In [7]:
from utils.mypytable import MyPyTable 
import utils.plot_utils as plot_utils
import importlib
import utils.myclassifiers
importlib.reload(utils.myclassifiers)
from utils.myclassifiers import MyDecisionTreeClassifier
from utils.myclassifiers import MyKNeighborsClassifier
from utils.myclassifiers import MyDummyClassifier
from utils.myclassifiers import MyNaiveBayesClassifier
from utils.myrandomforestclassifier import MyRandomForestClassifier
import random

import utils.myevaluation
importlib.reload(utils.myevaluation)
import utils.myevaluation as myevaluation
from itertools import chain
import utils.myutils as utils
importlib.reload(utils)
importlib.reload(plot_utils)


<module 'utils.plot_utils' from '/home/Project/V2/DataScienceAlgsProject/utils/plot_utils.py'>

In [8]:
cont_table = MyPyTable()
disc_table = MyPyTable()

cont_table.load_from_file('data_continuous.csv')
disc_table.load_from_file('data_discretized.csv')

In [9]:
cont_target_vals = cont_table.get_column('morph')
cont_table.drop_cols(['morph'])

disc_target_vals = disc_table.get_column('morph')
disc_table.drop_cols(['morph'])

In [10]:
# Discretized
X_train_idx_folds, X_test_idx_folds = myevaluation.stratified_kfold_cross_validation(
    disc_table.data, disc_target_vals, n_splits=7)

X_train = [[disc_table.data[idx] for idx in fold] for fold in X_train_idx_folds]
y_train = [[disc_target_vals[idx] for idx in fold]
            for fold in X_train_idx_folds]
X_test = [[disc_table.data[idx] for idx in fold] for fold in X_test_idx_folds]
y_test = [[disc_target_vals[idx] for idx in fold]
            for fold in X_test_idx_folds]

knn = MyKNeighborsClassifier(3)
dummy = MyDummyClassifier()
# bayes = MyNaiveBayesClassifier()
dec_tree = MyDecisionTreeClassifier()
forest = MyRandomForestClassifier()

classifier_names = ['dummy', 'dec_tree', 'forest']
models = [dummy, dec_tree, forest]
preds = [[] for _ in models]

forest.F = 3
forest.M = 50
forest.N = 21

for i in range(len(X_train)):
    for j in range(len(models)):
        models[j].fit(X_train[i], y_train[i])
        preds[j] += models[j].predict(X_test[i])

y_test = list(chain.from_iterable(y_test))

for i in range(len(preds)):
    myevaluation.cross_val_stats(classifier_names[i], y_test, preds[i])

----------
Classifier: dummy
----------
----------
Accuracy: 0.5103522452272116
----------
Error Rate: 0.48964775477278843
----------
Confusion Matrix: [[0, 1821], [0, 1898]]
----------
Classifier: dec_tree
----------
----------
Accuracy: 0.5579456843237429
----------
Error Rate: 0.4420543156762571
----------
Confusion Matrix: [[869, 952], [692, 1206]]
----------
Classifier: forest
----------
----------
Accuracy: 0.551492336649637
----------
Error Rate: 0.44850766335036296
----------
Confusion Matrix: [[784, 1037], [631, 1267]]


In [11]:
# Continuous 
X_train_idx_folds, X_test_idx_folds = myevaluation.stratified_kfold_cross_validation(
    cont_table.data, cont_target_vals, n_splits=7)

X_train = [[cont_table.data[idx] for idx in fold] for fold in X_train_idx_folds]
y_train = [[cont_target_vals[idx] for idx in fold]
            for fold in X_train_idx_folds]
X_test = [[cont_table.data[idx] for idx in fold] for fold in X_test_idx_folds]
y_test = [[cont_target_vals[idx] for idx in fold]
            for fold in X_test_idx_folds]

# knn = MyKNeighborsClassifier(3)
dummy = MyDummyClassifier()
bayes = MyNaiveBayesClassifier()
# dec_tree = MyDecisionTreeClassifier()

classifier_names = ['dummy', 'bayes']
models = [dummy, bayes]
preds = [[] for _ in models]

for i in range(len(X_train)):
    # print('i', i)
    for j in range(len(models)):
        # print('j', j)
        models[j].fit(X_train[i], y_train[i])
        preds[j] += models[j].predict(X_test[i])

y_test = list(chain.from_iterable(y_test))

for i in range(len(preds)):
    if classifier_names[i] == 'bayes':
        preds[i] = bayes.convert_idxs_to_class_labels(y_test, preds[i])
        myevaluation.cross_val_stats(classifier_names[i], y_test, preds[i])
    else:
        myevaluation.cross_val_stats(classifier_names[i], y_test, preds[i])

----------
Classifier: dummy
----------
----------
Accuracy: 0.526129108536297
----------
Error Rate: 0.473870891463703
----------
Confusion Matrix: [[0, 2004], [0, 2225]]
----------
Classifier: bayes
----------
----------
Accuracy: 0.5568692362260582
----------
Error Rate: 0.4431307637739418
----------
Confusion Matrix: [[1318, 686], [1188, 1037]]


In [12]:
# Continuous kNN down selection 
utils.randomize_in_place(cont_table.data, cont_target_vals)
cont_table_copy = cont_table.data.copy()
cont_table_copy = cont_table_copy[:2001]
X_train_idx_folds, X_test_idx_folds = myevaluation.stratified_kfold_cross_validation(
    cont_table_copy, cont_target_vals, n_splits=7)

X_train = [[cont_table_copy[idx] for idx in fold] for fold in X_train_idx_folds]
y_train = [[cont_target_vals[idx] for idx in fold]
            for fold in X_train_idx_folds]
X_test = [[cont_table_copy[idx] for idx in fold] for fold in X_test_idx_folds]
y_test = [[cont_target_vals[idx] for idx in fold]
            for fold in X_test_idx_folds]

knn = MyKNeighborsClassifier(5)
# dummy = MyDummyClassifier()
# bayes = MyNaiveBayesClassifier()
# dec_tree = MyDecisionTreeClassifier()

classifier_names = ['dummy', 'knn']
models = [dummy, knn]
preds = [[] for _ in models]

for i in range(len(X_train)):
    # print('i', i)
    for j in range(len(models)):
        # print('j', j)
        models[j].fit(X_train[i], y_train[i])
        preds[j] += models[j].predict(X_test[i])

y_test = list(chain.from_iterable(y_test))

for i in range(len(preds)):
    myevaluation.cross_val_stats(classifier_names[i], y_test, preds[i])

----------
Classifier: dummy
----------
----------
Accuracy: 0.5322338830584707
----------
Error Rate: 0.46776611694152925
----------
Confusion Matrix: [[0, 936], [0, 1065]]
----------
Classifier: knn
----------
----------
Accuracy: 0.6721639180409795
----------
Error Rate: 0.3278360819590205
----------
Confusion Matrix: [[535, 401], [255, 810]]
